In [1]:
import os
os.chdir('../')

In [2]:
import cv2
import numpy as np
from metrics import Metrics
from data_pipeline import Pipeline
from model import Object_Detection_Model 

In [3]:
# Paths to the model files
yolov3_weights = 'models/lpr-yolov3.weights'
yolov3_cfg = 'models/lpr-yolov3.cfg'
yolov3_tiny_weights = 'models/lpr-yolov3-tiny.weights'
yolov3_tiny_cfg = 'models/lpr-yolov3-tiny.cfg'

# Load the models
yolov3_model = cv2.dnn.readNetFromDarknet(yolov3_cfg, yolov3_weights)
yolov3_tiny_model = cv2.dnn.readNetFromDarknet(yolov3_tiny_cfg, yolov3_tiny_weights)

In [4]:
dp = Pipeline()
images_paths = dp.extract("images")

selected_images = images_paths

In [ ]:
def predict(net, image_path, size=(416, 416), conf_threshold=0.5, nms_threshold=0.4):
        """
        Processes the image with the object detection model and returns predictions.

        :param image: The image to process.
        :return: Predictions made by the model.
        """
        # Load the image from the path
        image = cv2.imread(image_path)
    
        # Convert the preprocessed image to a blob
        blob = cv2.dnn.blobFromImage(image, 1/255.0, size, swapRB=True, crop=False)
        net.setInput(blob)

        # Get the output layer names
        layer_names = net.getLayerNames()
        output_layers_indices = net.getUnconnectedOutLayers().flatten()
        output_layers = [layer_names[i - 1] for i in output_layers_indices]

        # Forward pass to get output of the output layers
        outputs = net.forward(output_layers)
    

        class_ids = []
        confidences = []
        boxes = []

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > conf_threshold:
                    center_x, center_y, width, height = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)

                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([left, top, int(width), int(height)])

        # Apply Non-Maximum Suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

        final_predictions = []
        if len(indices) > 0:
        # Convert tuple of arrays to a list of indices if necessary
            if isinstance(indices[0], np.ndarray):
                indices = [idx[0] for idx in indices]
            # Otherwise, use indices directly if it's already in the desired format (list of integers)

            for i in indices:
                box = boxes[i]
                x, y, w, h = box[0], box[1], box[2], box[3]
                final_predictions.append((class_ids[i], confidences[i], (x, y, w, h)))

        return final_predictions

In [6]:
metrics = Metrics('analysis/reports/')
# Example: evaluating on a subset of your dataset
predictions_yolov3 = []
predictions_yolov3_tiny = []
ground_truth = []

for image in selected_images:  # Assume dataset is a list of (image_path, label) tuples
    pred_yolov3 = predict(yolov3_model, image, size=(416, 416), conf_threshold=0.5, nms_threshold=0.4)
    pred_yolov3_tiny = predict(yolov3_tiny_model, image, size=(416, 416), conf_threshold=0.5, nms_threshold=0.4)

    predictions_yolov3.append(pred_yolov3)
    predictions_yolov3_tiny.append(pred_yolov3_tiny)

In [7]:
import xml.etree.ElementTree as ET

def parse_xml_for_bounding_box(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    for obj in root.findall('object'):
        if obj.find('name').text == 'number_plate' or obj.find('name').text == 'num_plate':
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            xmax = int(obj.find('bndbox/xmax').text)
            ymax = int(obj.find('bndbox/ymax').text)
            return (xmin, ymin, xmax, ymax)
    return None

for image in selected_images:
    image = image.split('.')[0] + ".xml"
    ground_truth_bbox = parse_xml_for_bounding_box(image)
    ground_truth.append(ground_truth_bbox)
    
print(ground_truth)
print(predictions_yolov3)
print(predictions_yolov3_tiny)

[(1093, 645, 1396, 727), (134, 312, 301, 350), (31, 128, 139, 161), (164, 216, 316, 243), (813, 665, 1067, 724), (66, 166, 154, 197), (360, 174, 434, 195), (137, 249, 262, 290), (207, 174, 356, 287), (184, 220, 342, 257), (148, 250, 239, 320), (131, 130, 187, 144), (183, 211, 249, 227), (80, 364, 239, 402), (179, 256, 347, 290), (39, 129, 108, 157), (15, 33, 200, 126), (157, 226, 316, 278), (120, 161, 249, 202), (532, 636, 852, 706), (180, 216, 559, 314), (976, 669, 1284, 754), (647, 332, 736, 387), (311, 324, 506, 370), (342, 243, 494, 288), (11, 78, 246, 165), (165, 276, 328, 324), (149, 269, 399, 327), (24, 209, 142, 245), (58, 198, 221, 271), (301, 351, 504, 402), (419, 173, 513, 213), (198, 107, 349, 143), (111, 133, 320, 286), (795, 751, 1095, 840), (230, 188, 365, 219), (157, 197, 341, 247), (787, 473, 910, 513), (401, 456, 593, 493), (36, 64, 465, 168), (341, 954, 607, 1011), (1058, 660, 1351, 756), (346, 197, 423, 221), (150, 255, 348, 296), (66, 234, 174, 259), (115, 144, 524

In [8]:
flattened_pred_yolov3 = []
flattened_pred_yolov3_tiny = []
# For YOLOv3 predictions
for p in predictions_yolov3:
    # Check if the prediction list is not empty
    if p:
        # Assuming there's only one prediction per list as shown in your example
        single_pred = p[0]
        # Convert from (x, y, w, h) to (xmin, ymin, xmax, ymax)
        converted = (single_pred[2][0], single_pred[2][1], single_pred[2][0] + single_pred[2][2], single_pred[2][1] + single_pred[2][3])
        flattened_pred_yolov3.append(converted)
    else:
        flattened_pred_yolov3.append((0,0,0,0)) #No detection

# For YOLOv3-tiny predictions
for p in predictions_yolov3_tiny:
    # Check if the prediction list is not empty
    if p:
        # Assuming there's only one prediction per list as shown in your example
        single_pred = p[0]
        # Convert from (x, y, w, h) to (xmin, ymin, xmax, ymax)
        converted = (single_pred[2][0], single_pred[2][1], single_pred[2][0] + single_pred[2][2], single_pred[2][1] + single_pred[2][3])
        flattened_pred_yolov3_tiny.append(converted)
    else:
        flattened_pred_yolov3_tiny.append((0,0,0,0)) #No detection
        
        
print(flattened_pred_yolov3)
print(flattened_pred_yolov3_tiny)

[(1073, 649, 1424, 731), (121, 297, 309, 362), (15, 122, 152, 165), (150, 198, 323, 264), (781, 667, 1081, 739), (56, 163, 165, 201), (360, 172, 432, 196), (127, 248, 273, 296), (197, 173, 357, 286), (179, 204, 352, 259), (94, 242, 250, 320), (135, 126, 183, 147), (184, 208, 248, 230), (71, 363, 243, 403), (171, 254, 349, 297), (36, 126, 110, 159), (36, 0, 204, 144), (145, 222, 325, 283), (112, 167, 248, 201), (486, 629, 863, 715), (0, 0, 0, 0), (970, 665, 1280, 759), (642, 329, 740, 390), (314, 313, 502, 379), (346, 236, 488, 298), (1, 73, 247, 187), (176, 268, 320, 332), (101, 262, 415, 331), (16, 198, 153, 256), (38, 191, 235, 281), (274, 338, 512, 413), (419, 170, 513, 215), (198, 99, 347, 154), (70, 107, 379, 307), (0, 0, 0, 0), (222, 183, 367, 226), (138, 187, 353, 255), (803, 468, 905, 532), (386, 452, 591, 502), (-3, 64, 463, 178), (311, 932, 631, 1025), (1028, 648, 1375, 778), (342, 192, 425, 225), (134, 238, 362, 311), (66, 229, 175, 264), (67, 134, 540, 315), (28, 193, 194, 

In [9]:
model = Object_Detection_Model()
model.test(flattened_pred_yolov3, ground_truth, "yolov3")
model.test(flattened_pred_yolov3, ground_truth, "yolov3_tiny")

Report generated and saved to: analysis/reports/object_detection_report_yolov3.txt
Report generated and saved to: analysis/reports/object_detection_report_yolov3_tiny.txt


# Model Performance And Thought
Here are the performances of each model

Yolov3-tiny
Model Results:
Average IoU: 0.68
Precision: 0.70
Recall: 0.70
F1 Score: 0.70

Yolov3
Model Results:
Average IoU: 0.71
Precision: 0.89
Recall: 0.89
F1 Score: 0.89

As we can see we much better results with the base yolov3 instead of the tiny in every category. Due to this I will be utilizng yolov3 for the next part of the project